In [3]:
# package to get open air quality data
# !pip install py-openaq

In [4]:
import warnings
warnings.simplefilter('ignore')

In [91]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)  

import folium

from datetime import datetime, timedelta

import openaq

In [6]:
api = openaq.OpenAQ()

In [17]:
# get cities in india that have pollution stations
cities = api.cities(df=True, country = 'IN', limit=10000)
cities.sort_values('locations')

,country,name,city,count,locations
199,IN,Yamuna Nagar,Yamuna Nagar,531363,1
146,IN,Panchkula,Panchkula,899063,1
35,IN,Brajrajnagar,Brajrajnagar,590763,1
144,IN,Palwal,Palwal,18,1
141,IN,NOIDA,NOIDA,40530,1
...,...,...,...,...,...
43,IN,Chennai,Chennai,3311219,22
82,IN,Hyderabad,Hyderabad,5289505,36
83,IN,Imphal,Imphal,76770,39
129,IN,Mumbai,Mumbai,6375486,41


In [21]:
# get stations in delhi
pollution_stations = api.locations(city='Delhi', df=True, limit=1000)[['location', 'lastUpdated', 'parameters', 'countsByMeasurement', 'coordinates.latitude', 'coordinates.longitude']]
pollution_stations.head()

,location,lastUpdated,parameters,countsByMeasurement,coordinates.latitude,coordinates.longitude
0,"DTU, Delhi - CPCB",2022-09-30 23:45:00+00:00,"[pm25, so2, o3, no2, co, pm10]","[{'parameter': 'pm25', 'count': 140570}, {'parameter': 'so2', 'count': 132232}, {'parameter': 'o3', 'count': 142589}, {'parameter': 'no2', 'count': 143319}, {'parameter': 'co', 'count': 142630}, {'parameter': 'pm10', 'count': 142054}]",28.750050,77.111261
1,"ITO, Delhi - CPCB",2022-09-30 23:45:00+00:00,"[pm25, so2, pm10, o3, co, no2]","[{'parameter': 'pm25', 'count': 69745}, {'parameter': 'so2', 'count': 70549}, {'parameter': 'pm10', 'count': 57478}, {'parameter': 'o3', 'count': 70484}, {'parameter': 'co', 'count': 70470}, {'parameter': 'no2', 'count': 70507}]",28.628624,77.241060
2,US Diplomatic Post: New Delhi,2022-09-30 23:30:00+00:00,[pm25],"[{'parameter': 'pm25', 'count': 126878}]",28.635760,77.224450
3,"CRRI Mathura Road, Delhi - IMD",2022-09-30 23:15:00+00:00,"[o3, pm10, co, no2, pm25]","[{'parameter': 'o3', 'count': 129245}, {'parameter': 'pm10', 'count': 129140}, {'parameter': 'co', 'count': 129245}, {'parameter': 'no2', 'count': 129158}, {'parameter': 'pm25', 'count': 128120}]",28.551201,77.273574
4,"Lodhi Road, Delhi - IMD",2022-09-30 23:15:00+00:00,"[co, o3, pm10, no2, pm25]","[{'parameter': 'co', 'count': 113065}, {'parameter': 'o3', 'count': 112891}, {'parameter': 'pm10', 'count': 112955}, {'parameter': 'no2', 'count': 113044}, {'parameter': 'pm25', 'count': 112648}]",28.591825,77.227307


In [73]:
# get pollution stations data in delhi
pollution_data = api.measurements(city='Delhi', df=True, limit=1000)

In [74]:
# filter last 12 hours of data 
pollution_data = pollution_data.loc[(datetime.now() - timedelta(hours = 24)).strftime("%Y-%m-%d %H:%M:%S"):]
pollution_data.head()

,location,parameter,value,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude
date.local,,,,,,,,,
2022-10-01 05:15:00,"ITO, Delhi - CPCB",o3,32.3,b'\xc2\xb5g/m\xc2\xb3',IN,Delhi,2022-09-30 23:45:00+00:00,28.628624,77.241060
2022-10-01 05:15:00,"DTU, Delhi - CPCB",so2,9.9,b'\xc2\xb5g/m\xc2\xb3',IN,Delhi,2022-09-30 23:45:00+00:00,28.750050,77.111261
2022-10-01 05:15:00,"DTU, Delhi - CPCB",pm10,182.0,b'\xc2\xb5g/m\xc2\xb3',IN,Delhi,2022-09-30 23:45:00+00:00,28.750050,77.111261
2022-10-01 05:15:00,"DTU, Delhi - CPCB",pm25,47.0,b'\xc2\xb5g/m\xc2\xb3',IN,Delhi,2022-09-30 23:45:00+00:00,28.750050,77.111261
2022-10-01 05:15:00,"ITO, Delhi - CPCB",pm10,180.0,b'\xc2\xb5g/m\xc2\xb3',IN,Delhi,2022-09-30 23:45:00+00:00,28.628624,77.241060


In [75]:
pollution_data_12_hrs = pollution_data.groupby(by=['location', 'parameter', 'coordinates.latitude','coordinates.longitude']).mean('value').reset_index()
pollution_data_12_hrs

,location,parameter,coordinates.latitude,coordinates.longitude,value
0,"Alipur, Delhi - DPCC",co,28.815329,77.153010,760.000000
1,"Alipur, Delhi - DPCC",no2,28.815329,77.153010,28.880000
2,"Alipur, Delhi - DPCC",o3,28.815329,77.153010,2.300000
3,"Alipur, Delhi - DPCC",pm10,28.815329,77.153010,213.600000
4,"Alipur, Delhi - DPCC",pm25,28.815329,77.153010,92.000000
...,...,...,...,...,...
173,"Wazirpur, Delhi - DPCC",no2,28.699793,77.165453,37.200000
174,"Wazirpur, Delhi - DPCC",o3,28.699793,77.165453,7.411111
175,"Wazirpur, Delhi - DPCC",pm10,28.699793,77.165453,229.111111
176,"Wazirpur, Delhi - DPCC",pm25,28.699793,77.165453,102.888889


In [77]:
pollution_data_12_hrs = pollution_data_12_hrs.pivot(index=['location', 'coordinates.latitude', 'coordinates.longitude'], columns='parameter', values='value').fillna(0).reset_index()
pollution_data_12_hrs.head()

parameter,location,coordinates.latitude,coordinates.longitude,co,no2,o3,pm10,pm25,so2
0,"Alipur, Delhi - DPCC",28.815329,77.153010,760.000000,28.880000,2.3000,213.60000,92.000000,7.380000
1,"Ashok Vihar, Delhi - DPCC",28.695381,77.181665,1475.000000,27.900000,8.4500,213.75000,94.250000,4.075000
2,"Aya Nagar, Delhi - IMD",28.470691,77.109936,1811.250000,21.270000,9.7675,149.13125,-2.540000,0.000000
3,"CRRI Mathura Road, Delhi - IMD",28.551201,77.273574,4250.000000,7.250000,28.5900,152.31500,90.560000,0.000000
4,"DTU, Delhi - CPCB",28.750050,77.111261,864.444444,-101.866667,5.4000,158.00000,40.555556,9.988889


In [116]:
m = folium.Map(location=[pollution_data_12_hrs['coordinates.latitude'].mean(), pollution_data_12_hrs['coordinates.longitude'].mean()],
              zoom_start=12)

for i, r in pollution_data_12_hrs.iterrows():
    html = f"<table><tr ><th>Parameter</th><th>mgm3</th></tr><tr><td>co</td><td>{round(r.co,2)}</td></tr><tr><td>no2</td><td>{round(r.no2,2)}</td></tr><tr><td>o2</td><td>{round(r.o3,2)}</td></tr><tr><td>pm10</td><td>{round(r.no2,2)}</td></tr><tr><td>pm25</td><td>{round(r.pm25,2)}</td></tr><tr><td>so2</td><td>{round(r.so2,2)}</td></tr></table>" 
    iframe = folium.IFrame(html)
    popup = folium.Popup(iframe,
                     min_width=210,
                     max_width=300,
                     height=500)
    folium.Marker(
        location=[r['coordinates.latitude'], r['coordinates.longitude']],
        popup= popup,
        icon=folium.Icon(icon="cloud"),
        ).add_to(m)
m